# Using Qiskit QPUs on the QLM
Here is an exemple of how the Qiskit binders could be used to run a QLM circuit by using the IBM Quantum Experience.

## Writing a circuit

In [ ]:
from qat.lang.AQASM import Program, H

prog = Program()
reg = prog.qalloc(1)
prog.apply(H, reg[0])

job = prog.to_circ().to_job(nbshots=50)

## Selecting an IBM QPU
An IBMQ token needs to be provided in order to run any circuit on the IBMQ backends. [Creating an account](https://quantum-computing.ibm.com/) is very straightforward. This will allow you to copy your token and check the available backends.

In [ ]:
MY_IBM_TOKEN = "..."

In [ ]:
from qat.interop.qiskit import BackendToQPU
from qiskit import IBMQ

# we can select a backend from available IBMQ backends
# here we use the backend ibmq_armonk, it should not be too busy since it only has 1 qubit
qpu = BackendToQPU(token=MY_IBM_TOKEN, ibmq_backend="ibmq_armonk")
print("Selected backend:", qpu.backend)

## Running the job synchronously

In [ ]:
results = qpu.submit(job) # may take time as it is waiting for the backend to get the results
print(results)

## Running the job asynchronously
For running the job asynchronously, AsyncBackendToQPU shall be used.

In [ ]:
import time
from qat.interop.qiskit import AsyncBackendToQPU

async_qpu = AsyncBackendToQPU(token=MY_IBM_TOKEN, ibmq_backend="ibmq_armonk")

# here we get a QiskitJob object when submitting our QLM Job
qiskit_job = async_qpu.submit(job)
print(qiskit_job.status())

while not qiskit_job.result():
    # waiting for the job to be finished
    time.sleep(1)

print(qiskit_job.status())
results = qiskit_job.result()
print(results)

The perk of using an asynchronous job is to be able to store it and check its results later, this is accomplished by using the functions `QiskitJob.dump()` and `AsyncBackendToQPU.retrieve_job()`.

In [ ]:
qiskit_job = async_qpu.submit(job)

# job dumped
qiskit_job.dump('my_file')

# in order to retrieve the job, an AsyncBackendToQPU instance similar to the one used for submitting
# the QLM Job in the first place is needed
my_new_async_qpu = AsyncBackendToQPU(token=MY_IBM_TOKEN, ibmq_backend="ibmq_armonk")
retrieved_job = my_new_async_qpu.retrieve_job('my_file')

print("Original job's ID: ", qiskit_job.job_id(), "\nRetrieved job's ID:", retrieved_job.job_id())